In [11]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from PIL import Image

# Load image and convert to tensor
#testInput="testpattern-hd-1080.png"
testInput="1920x1080-full-hd-nature-landscape.jpg"
img = Image.open(testInput).convert("RGB")
img_tensor = T.ToTensor()(img).unsqueeze(0)  # shape [1, C, H, W]

# Crop height from 1080 → 960 (remove 60px top & bottom)
# Crop width from  1920 → 1280 (remove 320px left & rigth)
img_tensor = img_tensor[:, :, 60:1020, 320:1600]  # now shape [1, C, 960, 1280]

# Define mean filter convolution layer
channels = img_tensor.shape[1]
kernel_size = 3
stride = (2, 2)
padding = 1  # keep things centered

conv = nn.Conv2d(
    in_channels=channels,
    out_channels=1,
    kernel_size=kernel_size,
    stride=stride,
    padding=padding,
   # groups=channels,
    bias=False
)

# Set weights to mean filter
with torch.no_grad():
   # mean_kernel = torch.ones_like(conv.weight) / (kernel_size ** 2)
    mean_kernel = torch.zeros((1, channels, kernel_size, kernel_size))  # [out, in, kH, kW]
    luminance = torch.tensor([0.2989, 0.5870, 0.1140])
    for c in range(channels):
        mean_kernel[0, c, :, :] = luminance[c] /(kernel_size ** 2)  # spread weights evenly over 3x3
    conv.weight.copy_(mean_kernel)

# Apply convolution
downsampled = conv(img_tensor)  # should now be [1, 1, 480, 640]

# Save output
output = T.ToPILImage()(downsampled.squeeze(0).clamp(0, 1))
output.save("output_downsampled.jpg")
